In [1]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [2]:
from infrastructure.instrument_collection import instrumentCollection as ic

In [3]:
pair = "GBP_JPY"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [4]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [5]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2020-09-07 09:00:00+00:00,140.090,140.194,139.649,139.758
1,2020-09-07 13:00:00+00:00,139.755,140.076,139.598,140.041
2,2020-09-07 17:00:00+00:00,140.043,140.043,139.912,139.948
3,2020-09-07 21:00:00+00:00,139.960,140.020,139.746,139.786
4,2020-09-08 01:00:00+00:00,139.789,139.840,139.568,139.614


In [6]:
for ma in MA_LIST:    
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)

In [7]:
df_plot = df_ma.iloc[:500]

In [11]:
cp = CandlePlot(df_plot)

In [9]:
traces = [ f"MA_{x}" for x in MA_LIST ]

In [12]:
cp.show_plot()

In [13]:
cp.show_plot(line_traces=traces)

## decide buy/sell

In [14]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

In [15]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [16]:
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200
0,2020-10-22 13:00:00+00:00,137.066,137.324,137.014,137.262,136.76800,136.249350
1,2020-10-22 17:00:00+00:00,137.264,137.274,137.104,137.219,136.76378,136.236655
2,2020-10-22 21:00:00+00:00,137.215,137.341,136.788,136.912,136.75050,136.221010
3,2020-10-23 01:00:00+00:00,136.910,136.949,136.705,136.761,136.73320,136.205075
4,2020-10-23 05:00:00+00:00,136.758,137.164,136.669,136.937,136.72270,136.190830


In [17]:
df_an['DELTA'] = df_an.MA_50 - df_an.MA_200
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [18]:
def is_trade(row):
    if row.DELTA >= 0  and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0  and row.DELTA_PREV >= 0:
        return SELL
    return NONE

In [19]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [20]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [21]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
47,2020-11-03 10:00:00+00:00,136.028,136.240,135.903,136.089,135.99192,136.008620,-0.016700,0.012775,-1
81,2020-11-11 02:00:00+00:00,139.412,139.604,139.383,139.546,136.44624,136.422940,0.023300,-0.035460,1
738,2021-04-15 13:00:00+00:00,149.988,149.994,149.690,149.856,150.82828,150.902175,-0.073895,0.000280,-1
818,2021-05-04 21:00:00+00:00,151.845,152.005,151.760,151.959,150.96454,150.949005,0.015535,-0.027730,1
1022,2021-06-21 21:00:00+00:00,153.628,153.776,153.306,153.411,154.30770,154.323890,-0.016190,0.017955,-1


In [22]:
cp = CandlePlot(df_an.iloc[220:260])
cp.show_plot(line_traces=[MA_S, MA_L])

## calculate gain

In [23]:
ic.LoadInstruments("../data")

In [24]:
ic.instruments_dict[pair]

{'name': 'GBP_JPY', 'ins_type': 'CURRENCY', 'displayName': 'GBP/JPY', 'pipLocation': 0.01, 'tradeUnitsPrecision': 0, 'marginRate': 0.03}

In [25]:
ins_data = ic.instruments_dict[pair]

In [ ]:
df_trades.shape

In [26]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [27]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE'] 

In [28]:
df_trades.GAIN.sum()

-6889.500000000011

In [29]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [30]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])